# Function to compute the thermal angular spectrum

This notebooks contains the functions for computing the thermal angular spectrum, given the set of singular values and vectors of the absorption matrix evaluated at a specific frequency (or wave number) value.

Last time modified: 27.04.2025

## Necessary libraries:
- Numpy for vector functions, as well for general math functions
- Treams to compute T-matrices

For T-matrices, the notebook is set by default for the helicity basis. More information in https://tfp-photonics.github.io/treams/treams.html# 

In [1]:
import numpy as np
import treams as trm

### Physical constants

In [2]:
hbar = 1.05457182e-34 #Dirac's constant in Js
Kb = 1.380649e-23 #Boltzmann's constant in J/K
c0 = 299792458 #Speed of light in vaccuum in m/s

## Function to transform from the plane wave basis to the multipolar basis
The transformation from the state $|\mathbf{k}\, \lambda\rangle$ in the plane wave basis to the multipolar basis $|k\,j\,m\, \lambda\rangle$ is given by:

$$ |\mathbf{k}\, \lambda\rangle = \sum_{j=1}^\infty \sum_{m=-j}^j \sqrt{\frac{2j+1}{4\pi}}D^j_{m\lambda}(\theta, \phi,0) |k\,j\,m\, \lambda\rangle,$$
where $D^j_{m\lambda}(\theta, \phi,0)$ are the Wigner matrices elements. For computing purposes, the sum ober $j$ is truncated up to a value `jmax`.

`PW2SW` is the function that transforms a vector from the plane wave basis to the multipolar basis. It takes as arguments the polar angle `theta` and azimuthal angle `phi`, the helicity of the emitted radiation `lam`, and the maximum multipolar order `jmax`. Since the library Treams handles helicity values of 0 for -1 helicity, conversion to the value 1 is necessary. The function `SphericalWaveBasis.default(jmax)` provides the values of $j$ , $m$ and $\lambda$ for each element of the array given the value of `jmax`, which is necessary to know the corresponding values to transform. Since states with opposite polarization are orthogonal, only elements with the same polarization as the one of interest (`lam`) are converted. The function returns a vector with the coefficients, whose dimension is $2j_\text{max}(j_\text{max} + 2).$


In [3]:
def PW2SW(theta, phi, lam):
    coeff = []
    indices = trm.SphericalWaveBasis.default(1)
    for count in range(len(indices)):
        j, m, la = indices[count][1:]
        if la == 0:
            la = -1
        if lam != la:
            coeff.append(0.0)
        else: 
            coeff.append(np.sqrt((2*j+1)/(4*np.pi))*trm.special.cython_special.wignerd(j,m,la,phi,theta,0))
    return coeff

# Function to compute the product $D\cdot s$
The function `prodDs` calculates the product between the elements of the Wigner matrices $D^j_{m\lambda}$ and the vectors $|s\rangle$ (Eq. (11)). The function takes as parameters the vector of coefficients in the multipolar basis obtained from the PW2SW function (`Dvec`), and a singular vector s. Since the library Treams handles helicity values of 0 for -1 helicity, conversion to the value -1 is necessary. The function returns the squared scalar product value between the vectors $D$ and $|s\rangle$.

In [4]:
def prodDs(Dvec, vec, lam):
    indices = trm.SphericalWaveBasis.default(1)
    summ = 0
    for count in range(len(indices)): #Loop over all possible combinacions of j,m and λ
        j, m, la = indices[count][1:]
        if la == 0:
            la = -1
        if la == lam:
            summ += np.conjugate(Dvec[count])*vec[count]
    return abs(summ)**2

# Function for computing the thermal angular spectrum

The thermal angular spectrum is given by:

$$k\mathbb{E}\left\{ |\langle\mathbf{k}\,\lambda|\Phi_\text{thermal}\rangle|^2 \right\} =\frac{c_0}{2\pi \left(\exp\left(\frac{\hbar c_0 k }{k_B T}\right)-1\right)}\times \sum_sq_s^2(k)|\sum_{jm}\sqrt{\frac{2j+1}{4\pi}} {D^j}^*_{m\lambda}(\phi+\pi,\pi-\theta,0)\vec{s}_{jm\lambda}(k)|^2 $$
        
        
The function `KirchhoffsLaw` computes the thermal angular spectrum (Eq. (18)). This function uses the coefficients given by `PW2SW` by considering the direction $-\vec{k}$. This function takes as arguments the angles `theta` and `phi`, the helicity `lam` of the emitted radiation, the singular values `vals` and vectors `vecs`, respectively, the temperature `T`, and the maximum multipolar order `jmax`, and returns the value of the angular thermal spectrum multiplied by `k`, for having the correct units of photon density.


In [5]:
def KirchhoffsLaw(theta, phi, lam, kvals, vals, vecs, T):
    beta = (hbar*c0)/(Kb*T)
    if lam == 0:
        lam = -1
    Dvec = PW2SW(np.pi-theta, phi + np.pi, lam) #Coefficients of the plane wave in spherical basis
    Expvals = []
    for f in range(len(kvals)): #loop over the frequencies
        Expval = 0
        for i in range(len(vals[0])): #loop over the modes
            Expval += c0*vals[f][i]/(2*np.pi*kvals[f]*(np.exp(beta*kvals[f]) -1)) * prodDs(Dvec, vecs[f][i], lam)
        Expvals.append(Expval)
    return kvals*Expvals